In [ ]:
import requests
import pandas as pd

# Step 1: Define API Endpoint & Parameters
BLS_API_URL = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"Content-Type": "application/json"}

# Replace with the desired time series ID (e.g., Whole Milk: APU0000701111)
data_payload = {
    "seriesid": ["APU0000701111", 'APU0000701312', 'APU0000701322', 'APU0000702111','APU0000702212','APU0000702421', 'APU0000703111', 'APU0000703112', 'APU0000703113','APU0000703213','APU0000703311','APU0000FC2101','APU0000703511','APU0000703613','APU0000FC3101','APU0000703432','APU0000FC4101','APU0000704111','APU0000704211','APU0000704212','APU0000FD3101','APU0000704312','APU0000FD2101','APU0000FD4101','APU0000706111'],
    "startyear": "2015",
    "endyear": "2025"
}

item_names = {
    "APU0000701111": "Flour, white, all purpose, per lb. (453.6 gm)",
    "APU0000701312": "Rice, white, long grain, uncooked, per lb. (453.6 gm)",
    "APU0000701322": "Spaghetti and macaroni, per lb. (453.6 gm)",
    "APU0000702111": "Bread, white, pan, per lb. (453.6 gm)",
    "APU0000702212": "Bread, whole wheat, pan, per lb. (453.6 gm)",
    "APU0000702421": "Cookies, chocolate chip, per lb. (453.6 gm)",
    "APU0000703111": "Ground chuck, 100% beef, per lb. (453.6 gm)",
    "APU0000703112": "Ground beef, 100% beef, per lb. (453.6 gm)",
    "APU0000703113": "Ground beef, lean and extra lean, per lb. (453.6 gm)",
    "APU0000703213": "Chuck roast, USDA Choice, boneless, per lb. (453.6 gm)",
    "APU0000703311": "Round roast, USDA Choice, boneless, per lb. (453.6 gm)",
    "APU0000FC2101": "All Uncooked Beef Roasts, per lb. (453.6 gm)",
    "APU0000703511": "Steak, round, USDA Choice, boneless, per lb. (453.6 gm)",
    "APU0000703613": "Steak, sirloin, graded and ungraded, excluding USDA Prime and Choice, per lb. (453.6 gm)",
    "APU0000703432": "Beef for stew, boneless, per lb. (453.6 gm)",
    "APU0000FC3101": "All Uncooked Beef Steaks, per lb. (453.6 gm)",
    "APU0000FC4101": "All Uncooked Other Beef (Excluding Veal), per lb. (453.6 gm)",
    "APU0000704111": "Bacon, sliced, per lb. (453.6 gm)",
    "APU0000704211": "Chops, center cut, bone-in, per lb. (453.6 gm)",
    "APU0000704212": "Chops, boneless, per lb. (453.6 gm)",
    "APU0000FD3101": "All Pork Chops, per lb. (453.6 gm)",
    "APU0000704312": "Ham, boneless, excluding canned, per lb. (453.6 gm)",
    "APU0000FD2101": "All Ham (Excluding Canned Ham and Luncheon Slices), per lb. (453.6 gm)",
    "APU0000FD4101": "All Other Pork (Excluding Canned Ham and Luncheon Slices), per lb. (453.6 gm)",
    "APU0000706111": "Chicken, fresh, whole, per lb. (453.6 gm)",

}
response = requests.post(BLS_API_URL, json=data_payload, headers=headers)
data = response.json()

# Step 3: Extract & Convert Data to DataFrame
all_data = []

if "Results" in data:
    for series in data["Results"]["series"]:
        series_id = series["seriesID"]
        for item in series["data"]:
            all_data.append({
                "seriesid": series_id,
                "item_name": item_names.get(series_id, "Unknown Item"),
                "year_month": item["year"] + "-" + item["period"][1:],
                "value": item["value"]
            })

df_combined = pd.DataFrame(all_data)
df_combined.tail()

,seriesid,item_name,year_month,value
2972,APU0000706111,"Chicken, fresh, whole, per lb. (453.6 gm)",2015-05,1.483
2973,APU0000706111,"Chicken, fresh, whole, per lb. (453.6 gm)",2015-04,1.539
2974,APU0000706111,"Chicken, fresh, whole, per lb. (453.6 gm)",2015-03,1.545
2975,APU0000706111,"Chicken, fresh, whole, per lb. (453.6 gm)",2015-02,1.550
2976,APU0000706111,"Chicken, fresh, whole, per lb. (453.6 gm)",2015-01,1.546


In [ ]:
# Assuming df_combined is your DataFrame
# Convert 'value' column to numeric before pivoting
df_combined['value'] = pd.to_numeric(df_combined['value'], errors='coerce')

# Aggregate values for duplicate 'year_month' using a suitable method (e.g., mean)
df_pivoted = df_combined.pivot_table(
    index='year_month',
    columns='item_name',
    values='value',
    aggfunc='mean'  # or any other suitable aggregation function
)

df_pivoted

item_name,"All Ham (Excluding Canned Ham and Luncheon Slices), per lb. (453.6 gm)","All Other Pork (Excluding Canned Ham and Luncheon Slices), per lb. (453.6 gm)","All Pork Chops, per lb. (453.6 gm)","All Uncooked Beef Roasts, per lb. (453.6 gm)","All Uncooked Beef Steaks, per lb. (453.6 gm)","All Uncooked Other Beef (Excluding Veal), per lb. (453.6 gm)","Bacon, sliced, per lb. (453.6 gm)","Beef for stew, boneless, per lb. (453.6 gm)","Bread, white, pan, per lb. (453.6 gm)","Bread, whole wheat, pan, per lb. (453.6 gm)",...,"Flour, white, all purpose, per lb. (453.6 gm)","Ground beef, 100% beef, per lb. (453.6 gm)","Ground beef, lean and extra lean, per lb. (453.6 gm)","Ground chuck, 100% beef, per lb. (453.6 gm)","Ham, boneless, excluding canned, per lb. (453.6 gm)","Rice, white, long grain, uncooked, per lb. (453.6 gm)","Round roast, USDA Choice, boneless, per lb. (453.6 gm)","Spaghetti and macaroni, per lb. (453.6 gm)","Steak, round, USDA Choice, boneless, per lb. (453.6 gm)","Steak, sirloin, graded and ungraded, excluding USDA Prime and Choice, per lb. (453.6 gm)"
year_month,,,,,,,,,,,,,,,,,,,,,
2015-01,3.211,2.991,3.988,5.838,7.530,4.734,5.589,5.712,1.479,1.948,...,0.545,4.235,6.032,4.375,4.405,0.692,5.620,1.264,6.052,8.080
2015-02,3.221,2.928,3.962,5.844,7.569,4.684,5.472,5.552,1.435,2.024,...,0.525,4.238,6.184,4.403,4.428,0.689,5.657,1.319,6.180,8.194
2015-03,3.176,2.904,3.867,5.872,7.661,4.709,5.367,5.595,1.440,2.042,...,0.518,4.200,6.099,4.364,4.314,0.670,5.736,1.392,6.209,8.372
2015-04,2.971,2.836,3.813,5.864,7.684,4.702,5.211,5.654,1.454,2.008,...,0.509,4.231,6.105,4.323,4.114,0.670,5.644,1.337,6.157,8.329
2015-05,3.045,2.804,3.786,5.915,7.773,4.590,4.940,5.581,1.463,2.041,...,0.520,4.136,6.195,4.308,4.164,0.676,5.653,1.335,6.185,8.816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08,4.691,3.693,4.403,7.630,10.875,7.355,6.785,7.298,1.951,2.674,...,0.574,5.577,6.918,5.638,5.633,1.085,7.161,1.421,8.226,11.540
2024-09,4.595,3.719,4.328,7.582,10.882,7.359,6.955,7.217,1.976,2.748,...,0.573,5.670,7.223,5.576,5.630,1.078,7.244,1.431,8.148,11.787
2024-10,4.660,3.703,4.273,7.368,10.565,7.109,6.869,7.177,1.937,2.833,...,0.566,5.588,7.220,5.586,5.670,1.056,7.008,1.445,8.154,11.497


In [ ]:
df_pivoted.columns

Index(['All Ham (Excluding Canned Ham and Luncheon Slices), per lb. (453.6 gm)',
       'All Other Pork (Excluding Canned Ham and Luncheon Slices), per lb. (453.6 gm)',
       'All Pork Chops, per lb. (453.6 gm)',
       'All Uncooked Beef Roasts, per lb. (453.6 gm)',
       'All Uncooked Beef Steaks, per lb. (453.6 gm)',
       'All Uncooked Other Beef (Excluding Veal), per lb. (453.6 gm)',
       'Bacon, sliced, per lb. (453.6 gm)',
       'Beef for stew, boneless, per lb. (453.6 gm)',
       'Bread, white, pan, per lb. (453.6 gm)',
       'Bread, whole wheat, pan, per lb. (453.6 gm)',
       'Chicken, fresh, whole, per lb. (453.6 gm)',
       'Chops, boneless, per lb. (453.6 gm)',
       'Chops, center cut, bone-in, per lb. (453.6 gm)',
       'Chuck roast, USDA Choice, boneless, per lb. (453.6 gm)',
       'Cookies, chocolate chip, per lb. (453.6 gm)',
       'Flour, white, all purpose, per lb. (453.6 gm)',
       'Ground beef, 100% beef, per lb. (453.6 gm)',
       'Ground beef, l

In [ ]:
df_pivoted.to_csv('bls_data_1.csv')

In [ ]:
# Step 1: Define API Endpoint & Parameters
import requests # import the requests library
import pandas as pd
BLS_API_URL2 = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers2 = {"Content-Type": "application/json"}

# Replace with the desired time series ID (e.g., Whole Milk: APU0000701111)
data_payload2 = {
    "seriesid": ['APU0000FF1101','APU0000708111','APU0000709112','APU0000710211','APU0000710212','APU0000710411','APU0000711211','APU0000711311','APU0000711411','APU0000711412','APU0000711415','APU0000712112','APU0000712211','APU0000FL2101','APU0000712311','APU0000713111','APU0000714233','APU0000715211','APU0000717311','APU0000718311','APU0000720111','APU0000720311'],
    "startyear": "2015",
    "endyear": "2025"
}

item_names2 = {
    "APU0000FF1101": "Chicken breast, boneless, per lb. (453.6 gm)",
    "APU0000708111": "Eggs, grade A, large, per doz.",
    "APU0000709112": "Milk, fresh, whole, fortified, per gal. (3.8 lit)",
    "APU0000710211": "American processed cheese, per lb. (453.6 gm)",
    "APU0000710212": "Cheddar cheese, natural, per lb. (453.6 gm)",
    "APU0000710411": "Ice cream, prepackaged, bulk, regular, per 1/2 gal. (1.9 lit)",
    "APU0000711211": "Bananas, per lb. (453.6 gm)",
    "APU0000711311": "Oranges, Navel, per lb. (453.6 gm)",
    "APU0000711411": "Grapefruit, per lb. (453.6 gm)",
    "APU0000711412": "Lemons, per lb. (453.6 gm)",
    "APU0000711415": "Strawberries, dry pint, per 12 oz. (340.2 gm)",
    "APU0000712112": "Potatoes, white, per lb. (453.6 gm)",
    "APU0000712211": "Lettuce, iceberg, per lb. (453.6 gm)",
    "APU0000FL2101": "Lettuce, romaine, per lb. (453.6 gm)",
    "APU0000712311": "Tomatoes, field grown, per lb. (453.6 gm)",
    "APU0000713111": "Orange juice, frozen concentrate, 12 oz. can, per 16 oz. (473.2 ml)",
    "APU0000714233": "Beans, dried, any type, all sizes, per lb. (453.6 gm)",
    "APU0000715211": "Sugar, white, all sizes, per lb. (453.6 gm)",
    "APU0000717311": "Coffee, 100%, ground roast, all sizes, per lb. (453.6 gm)",
    "APU0000718311": "Potato chips, per 16 oz.",
    "APU0000720111": "Malt beverages, all types, all sizes, any origin, per 16 oz. (473.2 ml)",
    "APU0000720311": "Wine, red and white table, all sizes, any origin, per 1 liter (33.8 oz)",
}
response2 = requests.post(BLS_API_URL2, json=data_payload2, headers=headers2)
data2 = response2.json()

# Step 3: Extract & Convert Data to DataFrame
all_data2 = []

if "Results" in data2:
    for series in data2["Results"]["series"]:
        series_id2 = series["seriesID"]
        for item in series["data"]:
            all_data2.append({
                "seriesid": series_id2,
                "item_name": item_names2.get(series_id2, "Unknown Item"),
                "year_month": item["year"] + "-" + item["period"][1:],
                "value": item["value"]
            })

df2_combined = pd.DataFrame(all_data2)
print(df2_combined.tail())

           seriesid                                          item_name  \
2482  APU0000720311  Wine, red and white table, all sizes, any orig...   
2483  APU0000720311  Wine, red and white table, all sizes, any orig...   
2484  APU0000720311  Wine, red and white table, all sizes, any orig...   
2485  APU0000720311  Wine, red and white table, all sizes, any orig...   
2486  APU0000720311  Wine, red and white table, all sizes, any orig...   

     year_month   value  
2482    2015-05  12.044  
2483    2015-04  12.335  
2484    2015-03  12.370  
2485    2015-02  12.370  
2486    2015-01  12.912  


In [ ]:
# Assuming df2_combined is your DataFrame # changed from df_combined
# Convert 'value' column to numeric before pivoting
df2_combined['value'] = pd.to_numeric(df2_combined['value'], errors='coerce')

# Aggregate values for duplicate 'year_month' using a suitable method (e.g., mean)
df2_pivoted = df2_combined.pivot_table(
    index='year_month',
    columns='item_name',
    values='value',
    aggfunc='mean'  # or any other suitable aggregation function
)

df2_pivoted

item_name,"American processed cheese, per lb. (453.6 gm)","Bananas, per lb. (453.6 gm)","Beans, dried, any type, all sizes, per lb. (453.6 gm)","Cheddar cheese, natural, per lb. (453.6 gm)","Chicken breast, boneless, per lb. (453.6 gm)","Coffee, 100%, ground roast, all sizes, per lb. (453.6 gm)","Eggs, grade A, large, per doz.","Grapefruit, per lb. (453.6 gm)","Ice cream, prepackaged, bulk, regular, per 1/2 gal. (1.9 lit)","Lemons, per lb. (453.6 gm)",...,"Malt beverages, all types, all sizes, any origin, per 16 oz. (473.2 ml)","Milk, fresh, whole, fortified, per gal. (3.8 lit)","Orange juice, frozen concentrate, 12 oz. can, per 16 oz. (473.2 ml)","Oranges, Navel, per lb. (453.6 gm)","Potato chips, per 16 oz.","Potatoes, white, per lb. (453.6 gm)","Strawberries, dry pint, per 12 oz. (340.2 gm)","Sugar, white, all sizes, per lb. (453.6 gm)","Tomatoes, field grown, per lb. (453.6 gm)","Wine, red and white table, all sizes, any origin, per 1 liter (33.8 oz)"
year_month,,,,,,,,,,,,,,,,,,,,,
2015-01,4.944,0.583,1.452,5.401,3.438,4.738,2.113,1.037,5.089,1.860,...,1.295,3.758,2.732,1.195,4.263,0.668,2.454,0.644,2.089,12.912
2015-02,4.598,0.591,1.484,5.480,3.511,4.910,2.088,1.026,4.955,1.868,...,1.287,3.496,2.734,1.185,4.298,0.646,2.090,0.659,1.849,12.370
2015-03,4.558,0.593,1.472,5.384,3.522,4.827,2.133,0.970,4.889,1.664,...,1.318,3.463,2.678,1.201,4.400,0.639,1.664,0.660,1.819,12.370
2015-04,4.543,0.597,1.510,5.384,3.504,4.990,2.065,1.067,4.791,1.750,...,1.315,3.397,2.662,1.165,4.412,0.645,1.852,0.663,1.847,12.335
2015-05,4.417,0.582,1.486,5.312,3.408,4.715,1.962,1.070,4.696,1.874,...,1.320,3.387,2.709,1.208,4.427,0.646,2.087,0.660,1.740,12.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08,4.841,0.612,1.711,5.763,3.952,6.311,3.204,1.586,6.357,1.959,...,1.792,4.044,4.273,1.728,6.246,1.006,2.344,1.004,1.996,13.958
2024-09,4.951,0.617,1.705,5.731,3.974,6.470,3.821,1.673,6.338,1.999,...,1.795,4.021,4.227,1.733,6.466,1.009,2.648,1.007,1.936,13.957
2024-10,5.059,0.619,1.678,5.844,4.037,6.644,3.370,NaN,6.295,2.056,...,1.745,4.041,4.247,1.805,6.517,0.998,2.629,1.003,1.980,13.768


In [ ]:
df2_pivoted.columns

Index(['American processed cheese, per lb. (453.6 gm)',
       'Bananas, per lb. (453.6 gm)',
       'Beans, dried, any type, all sizes, per lb. (453.6 gm)',
       'Cheddar cheese, natural, per lb. (453.6 gm)',
       'Chicken breast, boneless, per lb. (453.6 gm)',
       'Coffee, 100%, ground roast, all sizes, per lb. (453.6 gm)',
       'Eggs, grade A, large, per doz.', 'Grapefruit, per lb. (453.6 gm)',
       'Ice cream, prepackaged, bulk, regular, per 1/2 gal. (1.9 lit)',
       'Lemons, per lb. (453.6 gm)', 'Lettuce, iceberg, per lb. (453.6 gm)',
       'Lettuce, romaine, per lb. (453.6 gm)',
       'Malt beverages, all types, all sizes, any origin, per 16 oz. (473.2 ml)',
       'Milk, fresh, whole, fortified, per gal. (3.8 lit)',
       'Orange juice, frozen concentrate, 12 oz. can, per 16 oz. (473.2 ml)',
       'Oranges, Navel, per lb. (453.6 gm)', 'Potato chips, per 16 oz.',
       'Potatoes, white, per lb. (453.6 gm)',
       'Strawberries, dry pint, per 12 oz. (340.2 gm)',

In [ ]:
df2_pivoted.to_csv('bls_data_2.csv')

In [ ]:
import pandas as pd

# Load the two CSV files into DataFrames
df1 = pd.read_csv('bls_data_1.csv')
df2 = pd.read_csv('bls_data_2.csv')

# Merge the two DataFrames based on the 'year_month' column
merged_df = pd.merge(df1, df2, on='year_month', how='outer')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_bls_data.csv', index=False)

In [ ]:
# prompt: save merged bls to computer

from google.colab import files
files.download('merged_bls_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>